In [1]:
!gdown -q 1T6-AojJ-2ZwCR4X_SMeA5xmek7JLtolv
!unzip -q TruckDataset.zip

In [26]:
!pip -q install torchview
!pip -q install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.7 MB/s eta 0:00:00


In [28]:
import os
import cv2
import torch
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from torchview import draw_graph
from ultralytics import YOLO

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_path = "/content/Dataset/"

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [41]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def read_image(img_path, yolo=False):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    img = transform(img)
    img = img.to(torch.float32)
    img = img.to(device)
    return img

def make_prediction(model, img):
    if type(model) == YOLO:
        results = model.predict(img, save=False, verbose=False)
        return len(results[0].boxes)

    with torch.no_grad():
        outputs = model(img.unsqueeze(0))
        return outputs[0][0].item()


def evaluate(model, yolo=False):
    model.eval()
    y_true = []
    y_pred = []
    print("Real value - Rounded prediction - Original prediction")
    with open(dataset_path+"/test/number_of_axles_test.txt", 'r') as f:
        for i in f.readlines():
            img_path, label = i.split(",")
            label = int(label)

            y_true.append(label)

            img = os.path.join(dataset_path+"test/images", img_path)
            if not yolo:
                img = read_image(img)

            prediction = make_prediction(model, img)
            round_prediction = round(prediction)
            y_pred.append(round_prediction)

            if round_prediction != label:
                print(f"{label}: {round_prediction} - {prediction}")

    precision, recall, f1score, _ = precision_recall_fscore_support(y_true, y_pred, average="macro")
    accuracy = accuracy_score(y_true, y_pred)

    print("")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1score}")
    print(f"Accuracy: {accuracy}")



# Inception

In [19]:
class InceptionModel(nn.Module):
    def __init__(self):
        super(InceptionModel, self).__init__()

        self.model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
        self.model.aux_logits = False

        self.model.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = self.model(x)
        return x

inception = InceptionModel()
inception = inception.to(device)
inception.load_state_dict(torch.load("inception.pth"))

<ipython-input-19-cc6ada57671e>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inception.load_state_dict(torch.load("inception.pth"))


<All keys matched successfully>

In [22]:
evaluate(inception)

Real value - Rounded prediction - Original prediction
7: 6 - 5.939828395843506
5: 4 - 4.160600662231445
3: 4 - 3.604426860809326
3: 2 - 2.1354315280914307
4: 3 - 3.136538028717041
6: 5 - 4.732193946838379
7: 6 - 6.214473724365234
4: 2 - 2.334242820739746

Precision: 0.8796660532167778
Recall: 0.8519126854233238
F1 Score: 0.8613615476518702
Accuracy: 0.926605504587156


# ResNet50

In [24]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.model = models.resnet50(weights=(models.ResNet50_Weights.IMAGENET1K_V1))

        self.model.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = self.model(x)
        return x

resnet = ResNet()
resnet = resnet.to(device)
resnet.load_state_dict(torch.load("resnet50.pth"))

<ipython-input-24-596f25acd43e>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load("resnet50.pth"))


<All keys matched successfully>

In [25]:
evaluate(resnet)

Real value - Rounded prediction - Original prediction
5: 6 - 5.994332790374756
5: 6 - 6.022765159606934
6: 7 - 6.539203643798828
2: 3 - 2.660851001739502
3: 2 - 2.2596256732940674
3: 4 - 3.546790361404419
3: 4 - 3.6322436332702637
5: 6 - 5.630774974822998
6: 5 - 5.450976371765137
4: 3 - 3.3756537437438965
4: 3 - 3.497750997543335
3: 4 - 3.558598279953003
4: 5 - 4.8895416259765625
4: 3 - 3.0367188453674316

Precision: 0.7672982868195634
Recall: 0.7785079105291871
F1 Score: 0.7713121138406368
Accuracy: 0.8715596330275229


# Yolo

In [46]:
yolo = YOLO("best.pt")

In [42]:
evaluate(model=yolo, yolo=True)

Real value - Rounded prediction - Original prediction
2: 3 - 3
2: 4 - 4
3: 4 - 4
4: 5 - 5

Precision: 0.9280778323331514
Recall: 0.9497872340425532
F1 Score: 0.9373882441435634
Accuracy: 0.963302752293578


In [48]:
metrics = model.val(data="Dataset/teste.yaml")

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)


100%|██████████| 755k/755k [00:00<00:00, 18.7MB/s]
val: Scanning /content/Dataset/test/labels... 109 images, 0 backgrounds, 0 corrupt: 100%|██████████| 109/109 [00:00<00:00, 1092.73it/s]

val: New cache created: /content/Dataset/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:06<00:00,  1.08it/s]


                   all        109        398      0.994      0.995      0.995       0.77
Speed: 4.5ms preprocess, 25.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val2


In [56]:
print(f"mAP50-95 : {metrics.box.map}")
print(f"mAP50    : {metrics.box.map50}")
print(f"mAP75    : {metrics.box.map75}")
print(f"Precision: {metrics.box.p[0]}")
print(f"Recall   : {metrics.box.r[0]}")
print(f"F1 Score : {metrics.box.f1[0]}")


mAP50-95 : 0.7700400536318341
mAP50    : 0.994899115832326
mAP75    : 0.9399364363157161
Precision: 0.9944383623267529
Recall   : 0.9949748743718593
F1 Score : 0.9947065460050668
